In [ ]:
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

In [ ]:
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
# Configure
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

/content/darknet


In [ ]:
!make

In [ ]:
# Make a copy of yolov3.cfg
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
# Change lines in yolov3.cfg file
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 42000/' cfg/yolov3_training.cfg
!sed -i 's/steps=400000,450000/steps=33600,378000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=21@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=21@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=21@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=78@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=78@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=78@' cfg/yolov3_training.cfg

In [ ]:

!echo -e '1c\n2c\n5c\n10c\n20c\n50c\n1e\n50pf\n50pb\n1egf\n1egb\n1eb\n1cb\n2cb\n2eb\n10cb\n20cb\n50cb\n25p\n5cb\n2e' > data/obj.names
!echo -e 'classes= 21\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data



In [ ]:
!cp cfg/yolov3_training.cfg /mydrive/yolov3/yolov3_testing.cfg
!cp data/obj.names /mydrive/yolov3/classes.txt

In [ ]:
!mkdir data/obj
!unzip /mydrive/yolov3/images.zip -d data/obj

In [ ]:

import glob
images_list = glob.glob("data/obj/Final/*.jpg")
images_list.extend(glob.glob("data/obj/Final/*.JPG"))
images_list.extend(glob.glob("data/obj/Final/*.png"))
with open("data/train.txt", "w") as f:
    f.write("\n".join(images_list))
print(len(images_list))

3322


In [ ]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

In [ ]:
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show
# Uncomment below and comment above to re-start your training from last saved weights
#!./darknet detector train data/obj.data cfg/yolov3_training.cfg /mydrive/yolov3/yolov3_training_last.weights -dont_show